### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

IMPORTS

In [842]:
import pandas as pd 
import requests
import numpy as np

In [843]:
import json
import os
from dotenv import load_dotenv

GETTING AND CLEANING BICIMAD DATA

In [844]:
bicimad = pd.read_csv("../data/raw/bicimad_stations.csv", sep="\t")
bicimad

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


In [845]:
bicimad = bicimad.drop(["Unnamed: 0"], axis=1)

In [846]:
bicimad

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


In [847]:
split_bicimad = bicimad['geometry.coordinates'].str.strip('[]').str.split(',', expand=True).astype('float64')
split_bicimad.columns = ['longitude', 'latitude']
bicimad= pd.concat([bicimad,split_bicimad],axis=1)
bicimad

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,longitude,latitude
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]",-3.729970,40.438960
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]",-3.726990,40.443750
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]",-3.726930,40.443420
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]",-3.727295,40.448332


In [848]:
bicimad = bicimad.drop(["geometry.coordinates"], axis = 1)

In [849]:
bicimad = bicimad.drop(["light", "number", "activate", "no_available", "geometry.type"], axis = 1)

In [850]:
bicimad

,id,name,address,total_bases,dock_bikes,free_bases,reservations_count,longitude,latitude
0,1,1a - Puerta del Sol A,Puerta del Sol nº 1,30,0,0,0,-3.701834,40.417214
1,2,1b - Puerta del Sol B,Puerta del Sol nº 1,30,0,0,0,-3.701603,40.417313
2,3,2 - Miguel Moya,Calle Miguel Moya nº 1,24,0,0,0,-3.705842,40.420589
3,4,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,18,9,9,0,-3.706917,40.430294
4,5,4 - Malasaña,Calle Manuela Malasaña nº 5,24,23,1,0,-3.702587,40.428552
...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,24,3,19,0,-3.729970,40.438960
260,266,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,24,9,14,0,-3.726990,40.443750
261,267,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,24,12,10,0,-3.726930,40.443420
262,268,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,24,12,10,0,-3.727295,40.448332


GETTING AND CLEANING BICIPARK DATA

In [851]:
bicipark = pd.read_csv("../data/raw/bicipark_stations.csv", sep=";")
bicipark

,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates
0,0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,"[-3.6630679, 40.4846838]"
1,1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,"[-3.693897, 40.452973]"
2,2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,15,1,0,Point,"[-3.690786, 40.418146]"
3,3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,12,1,0,Point,"[-3.692879, 40.430361]"
4,4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,28001,10,3,0,0,Point,"[-3.692245, 40.420354]"
5,5,5,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,28036,13,13,1,0,Point,"[-3.678921, 40.472419]"
6,6,25,Bicipark Salamanca,Pl. del Marqués de Salamanca,Madrid,28006,5,-1,1,0,Point,"[-3.679172, 40.430315]"
7,7,67,Bicipark Villa de Paris,"Calle de Orellana, 23, 21",Madrid,28004,10,8,1,1,Point,"[-3.693698, 40.425608]"
8,8,1154,Bicipark Pitis,"Calle Pau Arroyo del Fresno, 181",Madrid,28049,10,7,1,0,Point,"[-3.7249142, 40.4959425]"
9,9,1169,Bicipark Aviación,Parking Aviación Española,Madrid,28024,10,10,1,0,Point,"[-3.78369522, 40.383491516]"


In [852]:
bicipark = bicipark.drop(["Unnamed: 0"], axis=1)

In [853]:
bicipark = bicipark.drop(["zip_code", "enabled", "reserved_places","geometry.type"], axis = 1)

In [854]:
split_bicipark = bicipark['geometry.coordinates'].str.strip('[]').str.split(',', expand=True).astype('float64')
split_bicipark.columns = ['longitude', 'latitude']
bicipark= pd.concat([bicipark,split_bicipark],axis=1)
bicipark

,stationId,stationName,address,city,total_places,free_places,geometry.coordinates,longitude,latitude
0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,10,9,"[-3.6630679, 40.4846838]",-3.663068,40.484684
1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,15,9,"[-3.693897, 40.452973]",-3.693897,40.452973
2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,16,15,"[-3.690786, 40.418146]",-3.690786,40.418146
3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,16,12,"[-3.692879, 40.430361]",-3.692879,40.430361
4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,10,3,"[-3.692245, 40.420354]",-3.692245,40.420354
5,5,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,13,13,"[-3.678921, 40.472419]",-3.678921,40.472419
6,25,Bicipark Salamanca,Pl. del Marqués de Salamanca,Madrid,5,-1,"[-3.679172, 40.430315]",-3.679172,40.430315
7,67,Bicipark Villa de Paris,"Calle de Orellana, 23, 21",Madrid,10,8,"[-3.693698, 40.425608]",-3.693698,40.425608
8,1154,Bicipark Pitis,"Calle Pau Arroyo del Fresno, 181",Madrid,10,7,"[-3.7249142, 40.4959425]",-3.724914,40.495942
9,1169,Bicipark Aviación,Parking Aviación Española,Madrid,10,10,"[-3.78369522, 40.383491516]",-3.783695,40.383492


In [855]:
bicipark = bicipark.drop(["geometry.coordinates"], axis= 1)

In [856]:
bicipark

,stationId,stationName,address,city,total_places,free_places,longitude,latitude
0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,10,9,-3.663068,40.484684
1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,15,9,-3.693897,40.452973
2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,16,15,-3.690786,40.418146
3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,16,12,-3.692879,40.430361
4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,10,3,-3.692245,40.420354
5,5,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,13,13,-3.678921,40.472419
6,25,Bicipark Salamanca,Pl. del Marqués de Salamanca,Madrid,5,-1,-3.679172,40.430315
7,67,Bicipark Villa de Paris,"Calle de Orellana, 23, 21",Madrid,10,8,-3.693698,40.425608
8,1154,Bicipark Pitis,"Calle Pau Arroyo del Fresno, 181",Madrid,10,7,-3.724914,40.495942
9,1169,Bicipark Aviación,Parking Aviación Española,Madrid,10,10,-3.783695,40.383492


GETTING AND CLEANING PLACES OF INTEREST DATA

In [857]:
places = requests.get("https://datos.madrid.es/egob/catalogo/300614-0-centros-educativos.json")

In [858]:
places = places.json()

In [859]:
places.keys()

dict_keys(['@context', '@graph'])

In [860]:
places = places["@graph"]

In [861]:
places = pd.json_normalize(places)

In [862]:
places = places.drop(["@id", "id", "relation", "address.district.@id", "address.area.@id", "address.locality", "address.postal-code", "organization.organization-desc", "organization.accesibility", "organization.schedule", "organization.services", "@type"], axis = 1)

In [863]:
places

,title,address.street-address,location.latitude,location.longitude,organization.organization-name
0,ABC Schule (Alemán),CALLE LEIZARAN 27,40.447932,-3.681382,ABC Schule (Alemán)
1,Academia Darío Estudio,CALLE GRAN VIA 63,40.422491,-3.709818,Academia Darío Estudio
2,AFI Escuela de Finanzas,CALLE MARQUES DE VILLAMEJOR 5,40.431247,-3.687559,AFI Escuela de Finanzas
3,Agencia Estatal Consejo Superior de Investigac...,CALLE SERRANO 115 BIS,40.441165,-3.687752,Agencia Estatal Consejo Superior de Investigac...
4,Alliance Française de Madrid,CUESTA SANTO DOMINGO 13,40.420151,-3.708855,Alliance Française de Madrid
...,...,...,...,...,...
1879,Universidad Pontificia Comillas. Facultad de D...,CALLE ALBERTO AGUILERA 23,40.429980,-3.711258,Universidad Pontificia Comillas. Facultad de D...
1880,Universidad Pontificia Comillas. Facultad de T...,CALLE UNIVERSIDAD DE COMILLAS 3 y 5,40.552126,-3.682802,Universidad Pontificia Comillas. Facultad de T...
1881,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,40.429980,-3.711258,Universidad Pontificia de Comillas. Facultad d...
1882,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,40.429980,-3.711258,Universidad Pontificia de Comillas. Facultad d...


In [880]:
places["location.latitude"].describe()

count    1879.000000
mean       40.426446
std         0.040332
min        40.332807
25%        40.394306
50%        40.428207
75%        40.453888
max        40.552126
Name: location.latitude, dtype: float64

DEFINING FUNCTION BICIMAD NEAREST STATIONS

In [865]:
#Global variables
places_lat_rad = np.radians(places['location.latitude'].to_numpy())
places_lon_rad = np.radians(places['location.longitude'].to_numpy())


In [866]:
def nearest_bicimad():
    data_list = []
    bicimad_lat_rad = np.radians(bicimad['latitude'].to_numpy())
    bicimad_lon_rad = np.radians(bicimad['longitude'].to_numpy())
    dlat = bicimad_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicimad_lon_rad[:, np.newaxis] - places_lon_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(bicimad_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicimad['name'].iloc[station_index]
        station_address = bicimad["address"].iloc[station_index]
        bikes_available = bicimad["dock_bikes"].iloc[station_index]
        station_id = bicimad["id"].iloc[station_index]
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"place": place, "place_address": place_address, "station_name": station, "bikes_available": bikes_available, "station_id" : station_id, "station_address": station_address,  "distance": min_distance})
        
    return pd.DataFrame(data_list)

In [867]:
nearest_bicimad()

,place,place_address,station_name,bikes_available,station_id,station_address,distance
0,ABC Schule (Alemán),CALLE LEIZARAN 27,147 - Doctor Arce 45,1,148,Avenida del Doctor Arce nº 45,146.60
1,Academia Darío Estudio,CALLE GRAN VIA 63,116b - Plaza de España B,0,124,Calle Princesa nº 5,145.70
2,AFI Escuela de Finanzas,CALLE MARQUES DE VILLAMEJOR 5,103 - Ortega y Gasset,0,108,Calle José Ortega y Gasset nº 4,130.77
3,Agencia Estatal Consejo Superior de Investigac...,CALLE SERRANO 115 BIS,250 - Serrano 113,8,258,Calle Serrano nº 113B,141.14
4,Alliance Française de Madrid,CUESTA SANTO DOMINGO 13,23 - Santo Domingo,23,26,Plaza de Santo Domingo nº 1,63.54
...,...,...,...,...,...,...,...
1879,Universidad Pontificia Comillas. Facultad de D...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,17,61,Calle Santa Cruz del Marcenado nº 24,124.87
1880,Universidad Pontificia Comillas. Facultad de T...,CALLE UNIVERSIDAD DE COMILLAS 3 y 5,217 - Pedro Rico,0,225,Calle Pedro Rico nº 4,7913.76
1881,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,17,61,Calle Santa Cruz del Marcenado nº 24,124.87
1882,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,17,61,Calle Santa Cruz del Marcenado nº 24,124.87


DEFINING FUNCTION BICIPARK NEAREST STATIONS

In [868]:
def nearest_bicipark():
    data_list_bicipark = []
    bicipark_lat_rad = np.radians(bicipark['latitude'].to_numpy())
    bicipark_lon_rad = np.radians(bicipark['longitude'].to_numpy())
    dlat = bicipark_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicipark_lon_rad[:, np.newaxis] - places_lon_rad
    
    a = np.sin(dlat / 2) ** 2 + np.cos(bicipark_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicipark['stationName'].iloc[station_index]
        station_address = bicipark["address"].iloc[station_index]
        spots_available = bicipark["free_places"].iloc[station_index]
        station_id = bicipark["stationId"].iloc[station_index]
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list_bicipark.append({"place": place, "place_address": place_address, "station_name": station, "station_id" : station_id, "spots_available" : spots_available, "station_address": station_address,  "distance": min_distance})
        
    return pd.DataFrame(data_list_bicipark)

In [869]:
nearest_bicipark()

,place,place_address,station_name,station_id,spots_available,station_address,distance
0,ABC Schule (Alemán),CALLE LEIZARAN 27,Bicipark Orense,22,9,"Avenida General Perón, 27",1198.13
1,Academia Darío Estudio,CALLE GRAN VIA 63,Bicipark Canalejas,1238,10,-,1020.46
2,AFI Escuela de Finanzas,CALLE MARQUES DE VILLAMEJOR 5,Bicipark Almagro,12,12,"Calle Almagro, 11",460.95
3,Agencia Estatal Consejo Superior de Investigac...,CALLE SERRANO 115 BIS,Bicipark Almagro,12,12,"Calle Almagro, 11",1277.26
4,Alliance Française de Madrid,CUESTA SANTO DOMINGO 13,Bicipark Canalejas,1238,10,-,837.67
...,...,...,...,...,...,...,...
1879,Universidad Pontificia Comillas. Facultad de D...,CALLE ALBERTO AGUILERA 23,Bicipark Almagro,12,12,"Calle Almagro, 11",1556.24
1880,Universidad Pontificia Comillas. Facultad de T...,CALLE UNIVERSIDAD DE COMILLAS 3 y 5,Bicipark Pitis,1154,7,"Calle Pau Arroyo del Fresno, 181",7190.17
1881,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,Bicipark Almagro,12,12,"Calle Almagro, 11",1556.24
1882,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,Bicipark Almagro,12,12,"Calle Almagro, 11",1556.24


GETTING REAL TIME INFORMATION ABOUT BIKES WITH API

In [870]:
def get_token():
    load_dotenv('../.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [871]:
def get_available_bikes():
    load_dotenv('../.env')
    token = os.environ.get("access_token")
    url = f"https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

CLEANING THE NEW DATA

In [872]:
data_bikes = get_available_bikes()
bicimad_real_time = pd.DataFrame(data_bikes["data"])
bicimad_real_time

,activate,address,dock_bikes,free_bases,geofence,geometry,id,integrator,light,name,...,reservations_count,tipo_estacionPBSC,total_bases,virtualDelete,virtual_bikes,virtual_bikes_num,code_district,code_suburb,geofenced_capacity,bikesGo
0,1,"Avenida del Ensanche de Vallecas, 9,",19,8,"{'type': 'Polygon', 'coordinates': [[[-3.61311...","{'type': 'Point', 'coordinates': [-3.61253028,...",2190,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,1,"453 - Avenida del Ensanche de Vallecas, 9",...,0,FIXED,27,False,[],0,18,183,20.0,[]
1,1,"Paseo de la Chopera,33,Comunidad de Madrid España",22,1,"{'type': 'Polygon', 'coordinates': [[[-3.70098...","{'type': 'Point', 'coordinates': [-3.700399664...",2205,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,1,"267 - Paseo de la Chopera, 33",...,0,FIXED,23,False,[],0,02,023,20.0,[]
2,1,"Paseo de la Castellana nº 122,",1,22,"{'type': 'Polygon', 'coordinates': [[[-3.69138...","{'type': 'Point', 'coordinates': [-3.6908, 40....",2224,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,0,150 - Castellana frente a Hermanos Pinzón,...,0,FIXED,23,False,[],0,05,051,20.0,[]
3,1,"Paseo de la Castellana nº 164,Comunidad de Mad...",2,21,"{'type': 'Polygon', 'coordinates': [[[-3.69000...","{'type': 'Point', 'coordinates': [-3.6894151, ...",2225,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,0,157- Castellana 164,...,0,FIXED,23,False,[],0,05,055,20.0,[]
4,1,"Guetaria , 84b,",19,5,"{'type': 'Polygon', 'coordinates': [[[-3.71628...","{'type': 'Point', 'coordinates': [-3.71569102,...",2226,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,1,395b - 395 - Guetaria 84b,...,0,FIXED,24,False,[],0,12,121,20.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,1,"Calle del Sorbe , 20,",15,9,"{'type': 'Polygon', 'coordinates': [[[-3.71379...","{'type': 'Point', 'coordinates': [-3.71320917,...",2070,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,2,"357 - Calle del Sorbe , 20",...,0,FIXED,24,False,[],0,11,111,20.0,[]
610,1,"Avenida del Mediterráneo nº 19,",15,9,"{'type': 'Polygon', 'coordinates': [[[-3.67569...","{'type': 'Point', 'coordinates': [-3.675102, 4...",1478,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,2,76 - Puerta de Mariano de Cavia,...,0,FIXED,24,False,[],0,03,036,20.0,[]
611,0,"Calle Francisco Balseiro nº 1,",0,0,"{'type': 'Polygon', 'coordinates': [[[-3.71111...","{'type': 'Point', 'coordinates': [-3.710525, 4...",1643,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,3,237 - Pablo Iglesias,...,0,FIXED,0,False,"[16045, 12956]",2,06,061,0.0,[]
612,1,"237- calle Francisco Balseiro,Comunidad de Mad...",0,0,"{'type': 'Polygon', 'coordinates': [[[-3.71086...","{'type': 'Point', 'coordinates': [-3.7102736, ...",2362,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,3,1664 - 237- calle Francisco Balseiro,...,0,FIXED,0,False,[],0,06,061,0.0,[]


In [873]:
bicimad_real_time[["longitude", "latitude"]] = bicimad_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))

In [874]:
bicimad_real_time = bicimad_real_time.drop(["geofence", "integrator", "reservations_count", "tipo_estacionPBSC", "virtual_bikes", "virtual_bikes_num", "code_district", "code_suburb", "bikesGo", "geometry", "light", "no_available", "number", "virtualDelete", "geofenced_capacity"], axis = 1)
bicimad_real_time

,activate,address,dock_bikes,free_bases,id,name,total_bases,longitude,latitude
0,1,"Avenida del Ensanche de Vallecas, 9,",19,8,2190,"453 - Avenida del Ensanche de Vallecas, 9",27,-3.612530,40.370440
1,1,"Paseo de la Chopera,33,Comunidad de Madrid España",22,1,2205,"267 - Paseo de la Chopera, 33",23,-3.700400,40.395000
2,1,"Paseo de la Castellana nº 122,",1,22,2224,150 - Castellana frente a Hermanos Pinzón,23,-3.690800,40.449100
3,1,"Paseo de la Castellana nº 164,Comunidad de Mad...",2,21,2225,157- Castellana 164,23,-3.689415,40.459137
4,1,"Guetaria , 84b,",19,5,2226,395b - 395 - Guetaria 84b,24,-3.715691,40.369168
...,...,...,...,...,...,...,...,...,...
609,1,"Calle del Sorbe , 20,",15,9,2070,"357 - Calle del Sorbe , 20",24,-3.713209,40.389870
610,1,"Avenida del Mediterráneo nº 19,",15,9,1478,76 - Puerta de Mariano de Cavia,24,-3.675102,40.407271
611,0,"Calle Francisco Balseiro nº 1,",0,0,1643,237 - Pablo Iglesias,0,-3.710525,40.451333
612,1,"237- calle Francisco Balseiro,Comunidad de Mad...",0,0,2362,1664 - 237- calle Francisco Balseiro,0,-3.710274,40.451682


In [875]:
def fnearest_bicimad_realtime(places, df):
    data_list = []
    df = df.loc[df["dock_bikes"] > 0]
    bicimad_real_time_lat_rad = np.radians(bicimad_real_time['latitude'].to_numpy())
    bicimad_real_time_lon_rad = np.radians(bicimad_real_time['longitude'].to_numpy())
    dlat = bicimad_real_time_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicimad_real_time_lon_rad[:, np.newaxis] - places_lon_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(bicimad_real_time_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicimad_real_time['name'].iloc[station_index]
        station_address = bicimad_real_time["address"].iloc[station_index]
        bikes_available = bicimad_real_time["dock_bikes"].iloc[station_index]
        station_id = bicimad_real_time["id"].iloc[station_index]
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"place": place, "place_address": place_address, "station_name": station, "bikes_available": bikes_available, "station_id" : station_id, "station_address": station_address,  "distance": min_distance})
        
    nearest_bicimad_realtime = pd.DataFrame(data_list)
    return nearest_bicimad_realtime

In [876]:
nearest_bicimad_realtime = fnearest_bicimad_realtime(places, bicimad_real_time)

In [877]:
nearest_bicimad_realtime

,place,place_address,station_name,bikes_available,station_id,station_address,distance
0,ABC Schule (Alemán),CALLE LEIZARAN 27,147 - Doctor Arce 45,6,1553,"Avenida del Doctor Arce nº 45,",175.64
1,Academia Darío Estudio,CALLE GRAN VIA 63,116b - Plaza de España B,2,1522,"Calle Princesa nº 5,",145.70
2,AFI Escuela de Finanzas,CALLE MARQUES DE VILLAMEJOR 5,103 - Ortega y Gasset,4,1506,"Calle José Ortega y Gasset nº 4,",105.45
3,Agencia Estatal Consejo Superior de Investigac...,CALLE SERRANO 115 BIS,250 - Serrano 113,2,1656,"Calle Serrano nº 113B,",174.10
4,Alliance Française de Madrid,CUESTA SANTO DOMINGO 13,23 - Santo Domingo,7,1426,"Plaza de Santo Domingo nº 1,",63.54
...,...,...,...,...,...,...,...
1879,Universidad Pontificia Comillas. Facultad de D...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,1,1461,"Calle Santa Cruz del Marcenado nº 24,",124.87
1880,Universidad Pontificia Comillas. Facultad de T...,CALLE UNIVERSIDAD DE COMILLAS 3 y 5,579 - Maria Tubau,1,2256,"579 - Maria Tubau,",4342.76
1881,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,1,1461,"Calle Santa Cruz del Marcenado nº 24,",124.87
1882,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,1,1461,"Calle Santa Cruz del Marcenado nº 24,",124.87


GETTING THE SAME DATAFRAME BUT FOR PARKING SPACES (IN CASE THE USER WANTS TO PARK THE BICIMAD BIKE)

In [878]:
def fnearest_bicimad_realtime_parking(places, df):
    data_list = []
    df = df[df["free_bases"] > 0]
    bicimad_real_time_lat_rad = np.radians(bicimad_real_time['latitude'].to_numpy())
    bicimad_real_time_lon_rad = np.radians(bicimad_real_time['longitude'].to_numpy())
    dlat = bicimad_real_time_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicimad_real_time_lon_rad[:, np.newaxis] - places_lon_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(bicimad_real_time_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicimad_real_time['name'].iloc[station_index]
        station_address = bicimad_real_time["address"].iloc[station_index]
        free_bases = bicimad_real_time["free_bases"].iloc[station_index]
        station_id = bicimad_real_time["id"].iloc[station_index]
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"place": place, "place_address": place_address, "station_name": station, "free_bases": free_bases, "station_id" : station_id, "station_address": station_address,  "distance": min_distance})
        
    return pd.DataFrame(data_list)

In [879]:
nearest_bicimad_realtime_parking = fnearest_bicimad_realtime_parking(places, bicimad_real_time)
nearest_bicimad_realtime_parking

,place,place_address,station_name,free_bases,station_id,station_address,distance
0,ABC Schule (Alemán),CALLE LEIZARAN 27,147 - Doctor Arce 45,17,1553,"Avenida del Doctor Arce nº 45,",175.64
1,Academia Darío Estudio,CALLE GRAN VIA 63,116b - Plaza de España B,16,1522,"Calle Princesa nº 5,",145.70
2,AFI Escuela de Finanzas,CALLE MARQUES DE VILLAMEJOR 5,103 - Ortega y Gasset,19,1506,"Calle José Ortega y Gasset nº 4,",105.45
3,Agencia Estatal Consejo Superior de Investigac...,CALLE SERRANO 115 BIS,250 - Serrano 113,25,1656,"Calle Serrano nº 113B,",174.10
4,Alliance Française de Madrid,CUESTA SANTO DOMINGO 13,23 - Santo Domingo,16,1426,"Plaza de Santo Domingo nº 1,",63.54
...,...,...,...,...,...,...,...
1879,Universidad Pontificia Comillas. Facultad de D...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,22,1461,"Calle Santa Cruz del Marcenado nº 24,",124.87
1880,Universidad Pontificia Comillas. Facultad de T...,CALLE UNIVERSIDAD DE COMILLAS 3 y 5,579 - Maria Tubau,22,2256,"579 - Maria Tubau,",4342.76
1881,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,22,1461,"Calle Santa Cruz del Marcenado nº 24,",124.87
1882,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,22,1461,"Calle Santa Cruz del Marcenado nº 24,",124.87
